In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer

# Loading Dataset

In [2]:
df = pd.read_csv('Datasets/100_dataset - 100.csv',  encoding = "ISO-8859-1")
df.head()

,title,token,pos,entity
0,Hadits Muslim Nomor 1,Dan,CONJ,O
1,Hadits Muslim Nomor 1,ia,PRON,O
2,Hadits Muslim Nomor 1,merupakan,VERB,O
3,Hadits Muslim Nomor 1,atsar,NOUN,O
4,Hadits Muslim Nomor 1,yang,PRON,O


In [3]:
df.isnull().sum()

title     0
token     0
pos       0
entity    0
dtype: int64

In [4]:
df.title.nunique(), df.token.nunique(), df.entity.nunique()

(100, 2276, 3)

In [5]:
df.groupby('entity').size()

entity
I-LOC       10
I-PER     2732
O        11127
dtype: int64

# Preprocess

In [6]:
X = df.drop('entity', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.entity.values

classes = np.unique(y)
classes = classes.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)

X_train.shape, y_train.shape

((9292, 2388), (9292,))

In [7]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['I-LOC', 'I-PER']

# Naive Bayes Model

In [8]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [9]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels = new_classes))

              precision    recall  f1-score   support

       I-LOC       0.00      0.00      0.00         2
       I-PER       0.89      0.98      0.93       887

   micro avg       0.88      0.97      0.93       889
   macro avg       0.44      0.49      0.47       889
weighted avg       0.89      0.97      0.93       889



In [10]:
nb.predict([X[37],X[38],X[39],X[40],X[41],X[42]])

array(['I-PER', 'I-PER', 'O', 'I-PER', 'I-PER', 'O'], dtype='<U5')

# SGD Model

In [11]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [12]:
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       I-LOC       0.00      0.00      0.00         2
       I-PER       0.82      0.98      0.89       887

   micro avg       0.82      0.97      0.89       889
   macro avg       0.41      0.49      0.45       889
weighted avg       0.82      0.97      0.89       889



/home/ghost/anaconda3/envs/Playgrounds/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [14]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent= 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['token'].values.tolist(), 
                                                           s['pos'].values.tolist(), 
                                                           s['entity'].values.tolist())]
        self.grouped = self.data.groupby('title').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
        

getter = SentenceGetter(df)
sentences = getter.sentences

In [104]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
#     print(sent[i])
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag+1': sent[i+1][1] if i < len(sent)-1 else '',
        'postag-1': sent[i-1][1] if i > 0 else '',
        'BOS': True if i == 0 else False,
        'EOS': True if i == len(sent)-1 else False,
#         'postag[:2]': postag[:2]
    }
#     if i > 0:
#         word1 = sent[i-1][0]
#         postag1 = sent[i-1][1]
#         features.update({
#             '-1:word.lower()': word1.lower(),
#             '-1:word.i
#             stitle()': word1.istitle(),
#             '-1:word.isupper()': word1.isupper(),
#             '-1:postag': postag1,
#             '-1:postag[:2]': postag1[:2],
#         })
#     else:
#         features['BOS'] = True
#     if i < len(sent)-1:
#         word1 = sent[i+1][0]
#         postag1 = sent[i+1][1]
#         features.update({
#             '+1:word.lower()': word1.lower(),
#             '+1:word.istitle()': word1.istitle(),
#             '+1:word.isupper()': word1.isupper(),
#             '+1:postag': postag1,
#             '+1:postag[:2]': postag1[:2],
#         })
#     else:
#         features['EOS'] = True
    return features

def sent2features(sent):  
#     print(sent)
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [105]:
# print(sentences[:1])
X = [x for s in sentences for x in sent2features(s)]
y = [x for s in sentences for x in sent2labels(s)]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [109]:
X[10]

{'bias': 1.0,
 'word.lower()': "qa'nabi",
 'word[-3:]': 'abi',
 'word[-2:]': 'bi',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'PROPN',
 'postag+1': 'ADV',
 'postag-1': 'PROPN',
 'BOS': False,
 'EOS': False}

In [107]:
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', SVC(gamma='auto'))
])

clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)), ('classifier', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [108]:
print(classification_report(y_test, clf.predict(X_test), labels=new_classes))

              precision    recall  f1-score   support

       I-LOC       0.00      0.00      0.00         4
       I-PER       0.82      0.84      0.83       907

   micro avg       0.82      0.84      0.83       911
   macro avg       0.41      0.42      0.42       911
weighted avg       0.82      0.84      0.83       911



/home/ghost/anaconda3/envs/Playgrounds/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [75]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))


              precision    recall  f1-score   support

       I-LOC       0.00      0.00      0.00         2
       I-PER       0.97      0.94      0.96      1081

   micro avg       0.97      0.94      0.95      1083
   macro avg       0.48      0.47      0.48      1083
weighted avg       0.97      0.94      0.95      1083



In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

In [ ]:
zz = df.groupby("title").apply(lambda s: [(w, p, t) for w, p, t in zip(s['token'].values.tolist(), 
                                                           s['pos'].values.tolist(), 
                                                           s['entity'].values.tolist())])

zz